onshell test (lcc randomized)
======

In [ ]:
from __future__ import print_function
import numpy as np
import sftrident.coordinate as coord
import sftrident.util as util
from IPython.display import display, clear_output

In [ ]:


def build_lcc_arr(N):
    ss = np.random.uniform(2.1,5,N)
    p1m = np.random.uniform(1e-5,3,N)
    p1x = np.random.uniform(-5,5,N)
    p1y = np.random.uniform(-5,5,N)
    p2m = np.random.uniform(1e-5,3,N)
    p2x = np.random.uniform(-5,5,N)
    p2y = np.random.uniform(-5,5,N)
    return util.cartesian_product(ss,p1m,p1x,p1y,p2m,p2x,p2y)


def transfrom(x,a,b):
    return a + (b-a)*x
    
    
def transfromAll(co):
    ss= np.array([3.353]*len(co))
    p1m = transfrom(co[:,0],1e-5,3)
    p1x = transfrom(co[:,1],-5,5)
    p1y = transfrom(co[:,2],-5,5)
    p2m = transfrom(co[:,3],1e-5,3)
    p2x = transfrom(co[:,4],-5,5)
    p2y = transfrom(co[:,5],-5,5)
    return np.column_stack([ss,p1m,p1x,p1y,p2m,p2x,p2y])
    

def build_rand_arr(n):
    return np.random.rand(n,6)


def print_fail(mom,coGrid):
    mass = mom*mom
    fail = (np.abs(mass - 1.0)>1e-4)
    #print(fail)
    print("\tmass:",np.asarray(mass))
    print("\tpoint:")
    print("\t\tlcc1:",coGrid.lccGrid1[0][fail][0],coGrid.lccGrid1[1][fail][0],coGrid.lccGrid1[2][fail][0])
    print("\t\tlcc2:",coGrid.lccGrid2[0][fail][0],coGrid.lccGrid2[1][fail][0],coGrid.lccGrid2[2][fail][0])
    print("\t\tlcc3:",coGrid.lccGrid3[0][fail][0],coGrid.lccGrid3[1][fail][0],coGrid.lccGrid3[2][fail][0])

    
task = 0
while True:
    lccArr = transfromAll(build_rand_arr(100000))

    #print(lccArr.shape)

    testMom = coord.momenta(lccArr,mode='batch')
    if not testMom.P1.isonshell(1.0):
        print("isonshell p1:",testMom.P1.isonshell(1.0))
        testCo = testMom.grid
        print_fail(testMom.P1,testCo)
        break
    if not testMom.P2.isonshell(1.0):
        print("p2:",testMom.P2.isonshell(1.0))
        testCo = testMom.grid
        print_fail(testMom.P2,testCo)
        break
    if not testMom.P3.isonshell(1.0):
        print("p3:",testMom.P3.isonshell(1.0))
        testCo = testMom.grid
        print_fail(testMom.P3,testCo)
        break
    task+=1
    clear_output(wait=True)
    print("%s rounds"%task)
        
    

In [ ]:
failCoord = np.array([3.353, 0.4520167767134499, 4.485385372483284, 4.494827775596903,0.04797565020975661, 3.2294204807779128, 2.0975609450728623])
failObj = coord.momenta(failCoord,mode='batch')
mom = failObj.P1
print(mom.isonshell(1.0))
print(mom*mom)

print(failObj.grid.lccGrid3)

testMom = coord.BuildFinalMom(1e-9, 5, -4,1.0)
print(np.array(testMom*testMom))

In [ ]:
import sftrident.qft as qft

def BuildFinalMom(pm,px,py,mass):
    #print pm
    pp = (px**2 + py**2 + mass**2)/(4.0*pm)
    a1=pp+pm
    print(a1)
    a2=(px**2 + py**2 + mass**2 + 4*pm**2)/(4*pm)
    print(a2)
    print(a1 - a2)
    
    a1=pp-pm
    print(a1)
    a2=(px**2 + py**2 + mass**2 - 4*pm**2)/(4*pm)
    print(a2)
    print(a1 - a2)
    print()
    return qft.MinkowskiVector(qft.parray([pp+pm,px,py,pp-pm]))


testMom = BuildFinalMom(7.5730767934711896e-06 ,-7.714805853261197 ,-6.592388720669765,1.0)
print(np.array(testMom*testMom))

In [ ]:
def physArea_cond_single(pm,px,py,m,atol=1e-4):
    """
    onshell condition for momenta in light cone coordinates
    """
    posPM = (pm>0.0)
    pmPos,pxPos,pyPos = [pm[posPM],px[posPM],py[posPM]]
    ppPos = (pxPos**2 + pyPos**2 + m**2)/(4.0*pmPos)
    print(ppPos)
    massSQ = (ppPos+pmPos)**2 - pxPos**2 - pyPos**2 - (ppPos-pmPos)**2
    print(massSQ)
    onshell = (np.abs(massSQ - m**2) < atol * m**2)
    print("%s"%massSQ[0])
    posPM[np.where(posPM)] = onshell
    return posPM

print(physArea_cond_single(np.array([7.5730767934711896e-06]), np.array([-7.714805853261197]), np.array([-6.592388720669765]),1.0))
print(physArea_cond_single(np.array([7.57307679e-06]), np.array([-7.71480585]), np.array([-6.59238872]),1.0))